In [1]:
import numpy as np
import pandas as pd
import math
import copy

In [2]:
varcost = np.array([[1,2,10,9,6,7,3],
           [2,9,0,7,3,6,10],
           [7,6,1,5,3,10,5],
           [6,5,10,2,6,3,6],
           [6,4,6,3,7,2,6]])

fixcost = [5, 7, 5, 6, 5]

(locations, customers) = varcost.shape


In [3]:
#solution encode: facility opening decision: 0 = close, 1 = open for each location
#for example: starting solution could be open all facilities
decision = np.ones(locations)

#for a given location-opening decision, there is an optimized location-customer allocation
allocation = np.zeros(varcost.shape)

In [4]:
#because location is uncapacitated so always transport all demand from cheapest location 
#=> transportation decision can be decoded as 0 (no transported) or 1 (transported)
#define a function to allocate with a given decision
def allocate (decision, varcost):
    varcost = np.array(varcost)
    x = np.zeros(varcost.shape)
    for j in range(customers):
        temp = [(decision[i] * varcost[i,j], i) for i in range(locations) if decision[i]!=0]  
        loc = np.argmin([k[0] for k in temp])
        x[temp[loc][1],j] = 1
    return x


In [5]:
#format of a solution
allocation = allocate(decision, varcost)
solution = [decision, allocation]

In [6]:
# a move = open a closed location (add move) or close an opened location (drop move)
def add_move(solution, varcost, i):
    new_sol = copy.deepcopy(solution)
    new_sol[0][i] = 1 #open a location
    new_sol[1] = allocate(new_sol[0], varcost) #re-allocation
    return new_sol

def drop_move(solution, varcost, i):
    new_sol = copy.deepcopy(solution)
    new_sol[0][i] = 0 #close a location
    new_sol[1] = allocate(new_sol[0], varcost) #re-allocation
    return new_sol

In [7]:
# define a function to calculate cost of a solution
def totalcost(solution, fixcost, varcost):
    fc = np.sum(np.multiply(solution[0], fixcost)) #element-wise multiplication
    vc = np.sum(np.multiply(solution[1], varcost))
    return fc+vc

In [8]:
# a set of neighborhood N(S) = {solutions obtained by applying a single local transformation (move, i.e. add or drop) to S}
# new solution must be feasible

def neighborhood(solution, fixcost, varcost):
    temp = []
    
    if sum(solution[0]) >=2: # feasible solution condition: at least 1 location is open
        i = np.random.randint(0,len(solution[0]))
        if solution[0][i] == 0: #location i is currently closed
            temp = add_move(solution, varcost, i)
        else: #location i is currently opened
            temp = drop_move(solution, varcost, i)
            
    else: #only 1 opened location in current solution
        close_loc = [ind for ind in range(len(solution[0])) if solution[0][ind]==0]
        i = np.random.choice(close_loc)
        temp = add_move(solution, varcost, i)
    
    delta = totalcost(solution, fixcost, varcost) - totalcost(temp, fixcost, varcost)
    
    return (temp, delta)

Help on built-in function choice:

choice(...) method of mtrand.RandomState instance
    choice(a, size=None, replace=True, p=None)
    
    Generates a random sample from a given 1-D array
    
            .. versionadded:: 1.7.0
    
    Parameters
    -----------
    a : 1-D array-like or int
        If an ndarray, a random sample is generated from its elements.
        If an int, the random sample is generated as if a were np.arange(a)
    size : int or tuple of ints, optional
        Output shape.  If the given shape is, e.g., ``(m, n, k)``, then
        ``m * n * k`` samples are drawn.  Default is None, in which case a
        single value is returned.
    replace : boolean, optional
        Whether the sample is with or without replacement
    p : 1-D array-like, optional
        The probabilities associated with each entry in a.
        If not given the sample assumes a uniform distribution over all
        entries in a.
    
    Returns
    --------
    samples : single item o

In [10]:
#main function
def SA (solution, cost, fixcost, varcost, initial=1, cool_factor=0.95, eval_times=10000, 
        TimeLimit = None, non_improvement = None, max_iteration = None, min_temp = None):

    best = cost
    best_solution = solution 
    best_iter = 0
    temperature = initial*best
    print(solution[0])
    print(best)
    
    #time limit condition
    from time import process_time 
    start_time = process_time()
     
    #max iteration condition
    iteration = 0
    non_iter = 0
    
    terminated = False
    while not terminated: #termination condition 
        
        improvement = False
        iteration += 1 
        
        #evaluation at given alpha
        for k in range(eval_times):
            (next_solution, delta) = neighborhood(solution, fixcost, varcost)
            #print(next_solution[0], delta, totalcost(next_solution, fixcost, varcost))
            
            if delta > 0 or (np.random.random() < math.exp(delta/temperature)): 
                #print('****', cost, delta)
                cost = cost - delta
                solution = next_solution
            if cost < best:
                best = cost
                best_solution = solution
                best_iter = iteration
                improvement = True
                print(best, '   ', temperature)
                
        temperature = cool_factor*temperature

        if (TimeLimit is not None): #time elapsed termination
            runtime = process_time() - start_time
            if (runtime >= TimeLimit): 
                print('Reach TimeLimit')
                terminated = True
                
        elif (non_improvement is not None): #max iteration without new best solution
            if improvement: 
                non_iter = 0
            else: non_iter += 1
            if non_iter >= non_improvement:
                print('Reach max number of iteration without new best solution')
                terminated = True
        
        elif (max_iteration is not None):
            if iteration >= max_iteration:
                print('Reach max number of iteration')
                terminated = True
                
        elif (min_temp is not None):
            if  temperature < min_temp : 
                print('Reach min temperature')
                terminated = True
                
        else: 
            if  temperature < (0.95**100) * initial*cost: #default option 
                print('Reach default min temperature')
                terminated = True
    
    print("Terminate")
    print(best_solution[0])
    print(best)

In [11]:
SA (solution, totalcost(solution,fixcost,varcost), fixcost, varcost, initial=1, cool_factor=0.95, eval_times=100, 
        TimeLimit = 60)

[1. 1. 1. 1. 1.]
41.0
36.0     41.0
30.0     41.0
Reach TimeLimit
Terminate
[1. 0. 1. 0. 1.]
30.0
